In [ ]:
import tensorflow as tf
import numpy as np
from helpers import optimize
from __future__ import print_function

def getSparcityPrior(inputX, lambda1=0.01, lambda2=0.01, optimizer='Adam', epochs=10000, learning_rate=0.1, print_step=50):
    tf.reset_default_graph()
    
    n_feat, n_sample = inputX.shape

    X = tf.placeholder(dtype=tf.float32, shape=[n_feat, n_sample], name='X')
    C = tf.Variable(tf.random_uniform([n_sample, n_sample], -1, 1), name='C')

    loss = X - tf.matmul(X, C)
    loss = tf.reduce_sum(tf.square(loss))

    # Create sparseness in C
    reg_lossC = tf.reduce_sum(abs(C))  # L1 loss for C

    # Force the entries in the diagonal of C to be zero
    reg_lossD = tf.trace(tf.square(C))

    cost = loss + lambda1 * reg_lossC + lambda2 * reg_lossD
    optimizer = optimize(cost, learning_rate, optimizer)
    
    # Optimizing the function
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        for i in xrange(epochs):
            sess.run(optimizer, feed_dict={X: inputX})
            loss = sess.run(cost, feed_dict={X: inputX})
            if i % print_step == 0:
                print('epoch {0}: global loss = {1}'.format(i, loss))
            C_val = sess.run(C)
        
        return C_val

In [7]:
import scipy.io as sio

# ourdata = sio.loadmat("/Users/xupeng.tong/Documents/Data/OriginalData/B_mean_2labels.mat")
ourdata = sio.loadmat("/Volumes/TONY/Regeneron/Data/OriginalData/B_mean_2labels.mat")

inputX = ourdata['X']
# inputX = normalize(inputX, axis=0)
inputY = ourdata['Y'][0,:]
columnNames = ourdata['columnNames']

In [9]:
from dsc import DeepSubspaceClustering
from sp import getSparcityPrior

c_init = np.full((inputX.shape[1],inputX.shape[1]), 0.1, dtype=np.float32)
np.fill_diagonal(c_init, 0)

C = getSparcityPrior(inputX, C_init=c_init, optimizer='Adam', lambda1=0.01, lambda2=50, epochs=500, learning_rate=0.1,print_step=1)
# dsc.train()

Calculating C ...
epoch 0: global loss = 2816743.5
epoch 1: global loss = 31432867840.0
epoch 2: global loss = 38608793600.0
epoch 3: global loss = 16658131968.0
epoch 4: global loss = 792171264.0
epoch 5: global loss = 4700127232.0
epoch 6: global loss = 16597424128.0
epoch 7: global loss = 19419502592.0
epoch 8: global loss = 11597244416.0
epoch 9: global loss = 2496568832.0
epoch 10: global loss = 239713872.0
epoch 11: global loss = 4868051456.0
epoch 12: global loss = 9819502592.0
epoch 13: global loss = 9656833024.0
epoch 14: global loss = 5109171712.0
epoch 15: global loss = 817597888.0
epoch 16: global loss = 325470720.0
epoch 17: global loss = 3061357568.0
epoch 18: global loss = 5502173696.0


KeyboardInterrupt: 

In [32]:
from dsc import DeepSubspaceClustering
from sp import getSparcityPrior
import tensorflow as tf
import numpy as np

test = np.random.randint(2, size=(100,100))
c_init = np.full((100,100), 0.05, dtype=np.float32)
# c_init = np.random.uniform(-0.1, 0.1, (20,20)).astype(np.float32)
np.fill_diagonal(c_init, 0)

keke = getSparcityPrior(test, C_init=None,optimizer='Adam', lambda1=0.001, lambda2=1000, epochs=5000, learning_rate=0.1,print_step=100)

Calculating C ...
epoch 0: global loss = 185711.6875
epoch 100: global loss = 1125.61914062
epoch 200: global loss = 421.432098389
epoch 300: global loss = 262.048248291
epoch 400: global loss = 196.122955322
epoch 500: global loss = 160.939468384
epoch 600: global loss = 141.654968262
epoch 700: global loss = 134.881256104
epoch 800: global loss = 119.022529602
epoch 900: global loss = 123.504333496
epoch 1000: global loss = 105.935211182
epoch 1100: global loss = 95.9519577026
epoch 1200: global loss = 91.5889511108
epoch 1300: global loss = 197.651321411
epoch 1400: global loss = 119.350341797
epoch 1500: global loss = 141.078125
epoch 1600: global loss = 496.251861572
epoch 1700: global loss = 184.521331787
epoch 1800: global loss = 187.798233032
epoch 1900: global loss = 372.561737061
epoch 2000: global loss = 237.840179443
epoch 2100: global loss = 122.25063324
epoch 2200: global loss = 257.447418213
epoch 2300: global loss = 450.617797852
epoch 2400: global loss = 281.617980957


In [33]:
keke

array([[  6.82652113e-04,   5.18126309e-01,  -2.08756462e-01, ...,
         -3.08756948e-01,  -5.03817081e-01,  -4.90358770e-01],
       [  5.17482579e-01,   7.30043103e-04,   1.06805682e-01, ...,
          3.70522141e-02,   3.93671334e-01,   3.57290089e-01],
       [ -6.17807031e-01,   2.95999974e-01,   2.40303285e-04, ...,
         -9.90450919e-01,  -8.04429233e-01,  -6.10158801e-01],
       ..., 
       [ -1.44622311e-01,   2.19002496e-02,  -1.84959263e-01, ...,
          1.72013789e-03,  -2.39778459e-01,  -1.65782824e-01],
       [ -9.42798495e-01,   6.81677163e-01,  -5.85100353e-01, ...,
         -1.05005646e+00,   2.62108369e-04,  -6.86959326e-01],
       [ -1.18184590e+00,   8.52450132e-01,  -5.39111853e-01, ...,
         -9.09550786e-01,  -9.13012743e-01,   2.40702560e-04]], dtype=float32)

In [34]:
test

array([[1, 0, 1, ..., 1, 1, 1],
       [0, 1, 1, ..., 0, 1, 1],
       [0, 0, 0, ..., 0, 1, 1],
       ..., 
       [0, 1, 0, ..., 0, 1, 1],
       [1, 0, 0, ..., 0, 1, 0],
       [1, 1, 1, ..., 1, 1, 0]])